# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [ ]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [ ]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

0    Евгения гр.ПМ19-1
1         Илья пм 20-4
2            Анна 20-3
dtype: object

3. Разбейте текст формулировки задачи 2 на слова.

---
## Лабораторная работа 6.1

### Форматирование строк

6\.1\.1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны.

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re
recipes = pd.read_csv('recipes_sample.csv')
rand_5 = recipes.loc[:, ['id', 'minutes']].sample(5)
ln = len(str(max(rand_5['id'])))
print(f'|{"id":^{ln + 4}}|{"minutes":^{ln + 5}}|')
print(f'|{"":-^{2 * ln + 10}}|')
for index, row in rand_5.iterrows():
    print(f'|{row["id"]:^{ln + 4}}|{row["minutes"]: ^{ln + 5}}|')

|    id    |  minutes  |
|----------------------|
|  124388  |    12     |
|  461103  |    15     |
|  427063  |    15     |
|   3304   |    15     |
|  100312  |    30     |


6\.1\.2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__).
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [1]:
# from bs4 import BeautifulSoup
# with open('steps_sample.xml') as f:
#     rec = BeautifulSoup(f, 'xml')
# rec

In [ ]:
def show_info(id):
    ans =''
    ans = f'"{recipes[recipes["id"] == id].name.values[0].title()}"\n'
    recipe = rec.find_all('recipe')
    for recipe in recipe:
        if recipe.id.text == str(id):
            num = 1
            for step in recipe.steps:
                if step != '\n':
                    ans += f'{num}. {step.text.capitalize()}\n'
                    num += 1
    ans += f'----------\nАвтор: {recipes[recipes["id"] == id]["contributor_id"].values[0]}\n'
    ans += f'Среднее время приготовления: {recipes[recipes["id"] == id]["minutes"].values[0]} минут'
    return ans
print(show_info(170895))

"Leeks And Parsnips  Sauteed Or Creamed"
1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an addit

In [ ]:
display(recipes[recipes['id'] == 170895])
p = re.compile(r'\d+ (?:hours|hour|minutes|minute)')

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
15754,leeks and parsnips sauteed or creamed,170895,27,8377,2006-05-31,21.0,this is good sauteed only or creamed. very eas...,9.0


## Работа с регулярными выражениями

6\.1\.3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [ ]:
string = show_info(25082)
p = re.compile(r'[\d]+ (?:hours|hour|minutes|minute)')
print(p.findall(string))
print(string)

['20 minutes', '10 minutes', '2 hours', '10 minutes', '20 minutes', '30 minutes']
"Basic Whole Wheat Bread"
1. Proof yeast in half a cup of the water , with about a teaspoon of flour- let it sit , covered , until it becomes bubbly& active
2. Combine the rest of the water with the yeast / water in a large bowl , and gradually add about half of the flour
3. Stir the mixture about 100 times in the same direction , until it is well-mixed and strands are beginning to form
4. Stir in the salt and oil
5. Add the remaining flour half a cup at a time , stirring well , until it is too stiff to stir with a spoon
6. Turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board
7. The dough should be springy and nice to work with
8. When it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly
9. After it has rested , knead the dough a few more times and pl

6\.1\.4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [ ]:
pd.options.display.max_colwidth = 100
p = re.compile(r'^[this][\w ]+, ?but')
# p = re.compile(r'^this[\w ]+, but')
strings = recipes['description'].str.match(p)
print(strings)
print(f"Количество рецептов: {strings.sum()}")
recipes[strings.fillna(False)].sample(3)[['description', 'id']]

0        False
1        False
2        False
3        False
4        False
         ...  
29995    False
29996    False
29997    False
29998     True
29999    False
Name: description, Length: 30000, dtype: object
Количество рецептов: 352


,description,id
6968,"this is a concoction for salmon that i came up with when i wanted salmon, but something differen...",414595
26417,"this recipe is based off of a cooking light recipe, but it is not pan fried at all. you could ea...",453725
3965,"this recipe is so good you will want to save it for company, but don't! your family deserves to...",57063


## Лабораторная работа 6.2

6\.2\.1\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [ ]:
string = show_info(72367)
# p = re.compile(r'(\d+)( )\/( )(\d+)')
p = re.compile(r'\d+ \/ \d')
print(re.sub(p, lambda s: s.group().replace(' ', ''), string))

"Bavarian Apple Cheese Cake Tart"
1. Mix butter , flour , 1/3 c
2. Sugar and 1-1/4 t
3. Vanilla
4. Press into greased 9" springform pan
5. Mix cream cheese , 1/4 c
6. Sugar , eggs and 1/2 t
7. Vanilla beating until fluffy
8. Pour over dough
9. Combine apples , 1/3 c
10. Sugar and cinnamon
11. Arrange on top of cream cheese mixture and sprinkle with almonds
12. Bake at 350 for 45-55 minutes , or until tester comes out clean
----------
Автор: 93006
Среднее время приготовления: 65 минут


### Сегментация текста

6\.2\.2\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [ ]:
import nltk
# nltk.download()

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
words = []
for recipe in rec.recipes.find_all('recipe'):
    for step in recipe.steps:
        for item in word_tokenize(step.text):
            if item.isalpha() and item.lower() not in words:
                words.append(item.lower())
print(len(words))

14926


6\.2\.3\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [ ]:
recipes['sentences'] = recipes['description'].apply(lambda x: sent_tokenize(str(x)))
recipes.sort_values(by='sentences', ascending=False, key=lambda a: a.str.len())[:5]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,sentences
18408,my favorite buttercream icing for decorating,334113,30,681465,2008-10-30,12.0,this wonderful icing is used for icing cakes and cookies as well as for borders and art work on ...,NaN,[this wonderful icing is used for icing cakes and cookies as well as for borders and art work on...
481,alligator claws avocado fritters with chipotle lime dip,287008,45,765354,2008-02-19,NaN,a translucent golden-brown crust allows the green of the avocado to be seen. the crispy exterio...,9.0,"[a translucent golden-brown crust allows the green of the avocado to be seen., the crispy exteri..."
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,"this is one of the best soups i've ever made and it is even worthy of company. so simple, yet r...",10.0,"[this is one of the best soups i've ever made and it is even worthy of company., so simple, yet ..."
16296,little bunny foo foo cake carrot cake with cream cheese frosti,316000,68,689540,2008-07-27,14.0,the first time i made this cake i grated a million pounds of carrots on a knucklebuster. then t...,NaN,"[the first time i made this cake i grated a million pounds of carrots on a knucklebuster., then ..."
6779,chocolate tea,205348,6,428824,2007-01-14,NaN,i wrote this because there are an astounding lack of chocolate tea recipes on the internet. \r\n...,NaN,"[i wrote this because there are an astounding lack of chocolate tea recipes on the internet., th..."


6\.2\.4\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
```
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [ ]:
from nltk.tokenize.toktok import ToktokTokenizer
toktok = ToktokTokenizer()
def text_tag(text):
    sent_text = []
    for wd in toktok.tokenize(text):
            if wd.isalpha():
                sent_text.append(wd)
    print(sent_text)
    tags = nltk.pos_tag(sent_text)
    print(tags)
    up = ''
    down = ''
    for el in tags:
        if (len(el[0]) > len(el[1])):
            up+= f'{el[1]:^{len(el[0])}} '
            down+=f'{el[0]} '
        else:
            up+= f'{el[1]} '
            down+=f'{el[0]:^{len(el[1])}} '
    res = up + '\n' + down
    return res
text = recipes.loc[recipes['id']==241106, 'name'].values[0]
print(text_tag(text))

['eggplant', 'steaks', 'with', 'chickpeas', 'feta', 'cheese', 'and', 'black', 'olives']
[('eggplant', 'JJ'), ('steaks', 'NNS'), ('with', 'IN'), ('chickpeas', 'NNS'), ('feta', 'VBP'), ('cheese', 'JJ'), ('and', 'CC'), ('black', 'JJ'), ('olives', 'NNS')]
   JJ     NNS    IN     NNS    VBP    JJ   CC   JJ    NNS   
eggplant steaks with chickpeas feta cheese and black olives 
